Taken from - https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/authorize_notebook_server.ipynb

In [ ]:
import ee

#ee.Initialize()

In [ ]:
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

In [ ]:
%%bash
earthengine authenticate --quiet

below taken from - https://github.com/Vizzuality/data_sci_tutorials/blob/master/work/authorize_notebook_server.ipynb

In [3]:
# Code to check if authorized to access Earth Engine.
import ipywidgets
import ee
import io
import os
import urllib
from IPython import display

# Define layouts used by the form.
row_wide_layout = ipywidgets.Layout(flex_flow="row nowrap", align_items="center", width="100%")
column_wide_layout = ipywidgets.Layout(flex_flow="column nowrap", align_items="center", width="100%")
column_auto_layout = ipywidgets.Layout(flex_flow="column nowrap", align_items="center", width="auto")

form_definition = {'form': None}
response_box = ipywidgets.HTML('')

def isAuthorized():
  try:
    ee.Initialize()
    test = ee.Image(0).getInfo()
  except:
    return False
  return True

def ShowForm(auth_status_button, instructions):
  """Show a form to the user."""
  form_definition['form'] = ipywidgets.VBox([
    auth_status_button,
    instructions,
    ipywidgets.VBox([response_box], layout=row_wide_layout)
  ], layout=column_wide_layout)
  display.display(form_definition.get('form'))

def ShowAuthorizedForm():
  """Show a form for a server that is currently authorized to access Earth Engine."""
  def revoke_credentials(sender):
    credentials = ee.oauth.get_credentials_path()
    if os.path.exists(credentials):
      os.remove(credentials)
    response_box.value = ''
    Init()
  
  auth_status_button = ipywidgets.Button(
    layout=column_wide_layout,
    disabled=True,
    description='The server is authorized to access Earth Engine',
    button_style='success',
    icon='check'
  )
  
  instructions = ipywidgets.Button(
    layout = row_wide_layout,
    description = 'Click here to revoke authorization',
    disabled = False,
  )
  instructions.on_click(revoke_credentials)
  
  ShowForm(auth_status_button, instructions)

def ShowUnauthorizedForm():
  """Show a form for a server that is not currently authorized to access Earth Engine."""

  auth_status_button = ipywidgets.Button(
    layout=column_wide_layout,
    button_style='danger',
    description='The server is not authorized to access Earth Engine',
    disabled=True
  )
    
  auth_link = ipywidgets.HTML(
    '<a href="{url}" target="auth">Open Authentication Tab</a><br/>'
    .format(url=ee.oauth.get_authorization_url()
    )
  )
    
  instructions = ipywidgets.VBox(
    [
      ipywidgets.HTML(
        'Click on the link below to start the authentication and authorization process. '
        'Once you have received an authorization code, use it to replace the '
        'REPLACE_WITH_AUTH_CODE in the code cell below and run the cell.'
      ),
      auth_link,
    ],
    layout=column_auto_layout
  )
  
  ShowForm(auth_status_button, instructions)
  
def Init():
  # If a form is currently displayed, close it.
  if form_definition.get('form'):
    form_definition['form'].close()
  # Display the appropriate form according to whether the server is authorized.
  if isAuthorized():
    ShowAuthorizedForm()
  else:
    ShowUnauthorizedForm()
    
Init()

In [6]:
auth_code = '4/pAGJHV-fINBz-in0_NcJNQiX9rpJN-R8mu_SAG-3KKS04ffOhIErE74'

response_box = ipywidgets.HTML('')
try:
  token = ee.oauth.request_token(auth_code.strip())
  ee.oauth.write_token(token)
  if isAuthorized():
    Init()
  else:
    response_box.value = '<font color="red">{0}</font>'.format(
      'The account was authenticated, but does not have permission to access Earth Engine.'
    )
except Exception as e:
  response_box.value = '<font color="red">{0}</font>'.format(e)
response_box

HTML(value='')

In [7]:
# Code to display an Earth Engine generated image.
from IPython.display import Image

url = ee.Image("CGIAR/SRTM90_V4").getThumbUrl({'min':0, 'max':3000})
Image(url=url)